In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
# import torchvision.datasets as dsets
# import torchvision.transforms as transforms
from torch.autograd import Variable
import pickle
import numpy as np
import random

BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCH = 300

# transform = transforms.Compose([
#     transforms.RandomSizedCrop(224),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean = [ 0.485, 0.456, 0.406 ],
#                          std  = [ 0.229, 0.224, 0.225 ]),
#     ])



# with open('./ck+.pkl', 'rb') as f:
#      data = pickle.load(f)
# print('load done')     

# index = [i for i in range(927)]
# random.shuffle(index)
# trainindex = index[:835]
# testindex = index[835:]

# # dic = {'angry':0, 'disgust':1, 'fear':2, 'happy':3, 'neutral':4, 'sad':5, 'surprise':6}
# dic = {'angry':0, 'disgusted':1, 'fearful':2, 'happy':3, 'sadness':4, 'surprised':5}
# for i in range(len(data[1])):
#     data[1][i] = dic[data[1][i]]
# testlabel = np.array(data[1])[testindex]
# trainlabel = np.array(data[1])[trainindex]
# testdata = np.array(data[3])[testindex]
# traindata = np.array(data[3])[trainindex]

# trainData = TensorDataset(torch.Tensor(traindata), torch.LongTensor(trainlabel))
# testData = TensorDataset(torch.Tensor(testdata), torch.LongTensor(testlabel))
# print('to tensor done')

# trainLoader = DataLoader(dataset=trainData, batch_size=BATCH_SIZE, shuffle=False)
# testLoader = DataLoader(dataset=testData, batch_size=1, shuffle=False)
# print('dataloader done')

with open('./raf_train.pkl', 'rb') as f:
     train_data = pickle.load(f)
print('load train done')

# with open('./raf_test.pkl', 'rb') as f:
#      test_data = pickle.load(f)
# print('load test done')


trainData = TensorDataset((torch.Tensor(train_data[2]))[:, :10], torch.LongTensor(train_data[0]))
# testData = TensorDataset(torch.Tensor(test_data[2]), torch.LongTensor(test_data[0]))
print('to tensor done')

trainLoader = DataLoader(dataset=trainData, batch_size=BATCH_SIZE, shuffle=True)
# testLoader = DataLoader(dataset=testData, batch_size=1, shuffle=False)
print('dataloader done')

KeyboardInterrupt: 

In [22]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
# import torchvision.datasets as dsets
# import torchvision.transforms as transforms
from torch.autograd import Variable
import pickle
import numpy as np
import random

with open('./raf_test.pkl', 'rb') as f:
     test_data = pickle.load(f)
print('load test done')

testData = TensorDataset((torch.Tensor(test_data[2]))[:, :10], torch.LongTensor(test_data[0]))
# testData = TensorDataset(torch.Tensor(test_data[2]), torch.LongTensor(test_data[0]))
print('to tensor done')


testLoader = DataLoader(dataset=testData, batch_size=1, shuffle=False)
print('dataloader done')

load test done
to tensor done
dataloader done


In [1]:
import torch
a = torch.FloatTensor([[[1,2],[1,2]],[[4,5],[4,5]],[[7,8],[7,8]]])
a.size()

torch.Size([3, 2, 2])

In [12]:
mean = torch.mean(torch.mean(a,axis=1),axis=1).reshape(3,1,1)
print(mean, mean.size())
a-mean

tensor([[[1.5000]],

        [[4.5000]],

        [[7.5000]]]) torch.Size([3, 1, 1])


tensor([[[-0.5000,  0.5000],
         [-0.5000,  0.5000]],

        [[-0.5000,  0.5000],
         [-0.5000,  0.5000]],

        [[-0.5000,  0.5000],
         [-0.5000,  0.5000]]])

In [19]:
checkpoint = torch.load('./RAFDB/resnetepoch99.pkl')
a = checkpoint['net']
new_check = OrderedDict()
for k in list(a.keys()):   
    newk = k.strip('module.') 
    # print(newk, k)
    a[newk] = a.pop(k)

In [25]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.conv0 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv1 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, 3, padding=1)
        self.conv5 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv6 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv7 = nn.Conv2d(64, 128, 1)
        self.conv8 = nn.Conv2d(128, 256, 1)
        self.BN1 = nn.BatchNorm2d(64)
        self.BN2 = nn.BatchNorm2d(128)
        self.BN3 = nn.BatchNorm2d(256)
        self.maxpool = nn.MaxPool2d(2, 2)
        self.avepool = nn.MaxPool2d(24, 24)
        self.fc1 = nn.Linear(256, 256)
        self.fc2 = nn.Linear(2560, 1024)
        self.fc3 = nn.Linear(1024, 7)
        self.dropout = nn.Dropout()

    def forward(self, x):
        # x.shape ==(-1,10,96,96,3)
        out = []
        for k in range(10):
            #print(k)
            f = x[:,k,:,:,:]
            f = f.permute(0,3,1,2) 
            #（-1，3，96，96）
            f1 = F.relu(self.BN1(self.conv0(f))) 
            #（-1，64，96，96）
            f2 = F.relu(self.BN1(self.conv1(f1)))
            f3 = self.maxpool(F.relu(f1 + self.BN1(self.conv2(f2))))
            residual1 = self.BN2(self.conv7(f3)) # (-1, 128, 48, 48)
            # (-1, 64, 48, 48)
            f4 = F.relu(self.BN2(self.conv3(f3))) 
            f5 = self.maxpool(F.relu(residual1 + self.BN2(self.conv4(f4))))
            residual2 = self.BN3(self.conv8(f5)) # (-1, 256, 24, 24)    
            # (-1, 128, 24, 24)
            f6 = F.relu(self.BN3(self.conv5(f5))) 
            f7 = F.relu(residual2 + self.BN3(self.conv6(f6)))
            f8 = self.avepool(f7)  
            # (-1, 256, 1, 1)
            f8 = f8.view(f8.size(0),-1)
            # (-1, 256)
            f9 = F.relu(self.fc1(f8))
            # (-1, 256)
            out.append(f9)
        out = torch.cat(out,axis=1) 
        #  (-1, 256*10)
        out = F.relu(self.fc2(self.dropout(out)))
        # (-1, 1024)
        out = F.softmax(self.fc3(out), dim=1)
        # (-1, 7)
        return out

In [35]:
from collections import OrderedDict
vgg16 = VGG16()
checkpoint = torch.load('./RAFDB/resnetepoch99.pkl')
# new_check = OrderedDict()
# for k,v in checkpoint.items():   
#     newk = k.strip('module.') 
#     # print(newk, k)
#     new_check[newk] = v
# checkpoint = torch.load('./RAFDB/vgg_dropout_epoch349.pkl')
vgg16.load_state_dict(checkpoint['net'])
vgg16.cuda()

# Test the model
vgg16.eval()
correct = 0
total = 0

for images, labels in testLoader:
    images = Variable(images).cuda()
    outputs = vgg16(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()

print('Test Accuracy of the model on the test images: %d %%' % (100 * correct / total))

RuntimeError: Error(s) in loading state_dict for VGG16:
	Unexpected key(s) in state_dict: "BN1.num_batches_track", "BN2.num_batches_track", "BN3.num_batches_track". 

In [26]:
vgg16

VGG16(
  (conv0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
  (conv8): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
  (BN1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (BN2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (BN3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, paddin

In [28]:
from torch.autograd import Variable
f = nn.Dropout(0.5)
a = Variable(torch.randn(20))
print(a)
print(f(a))

tensor([ 0.4742,  2.4438, -1.6943, -0.0654,  0.2538,  0.3002,  0.2470,  0.0383,
         0.6833, -1.2546,  1.0609, -0.8963,  0.8332,  2.6125, -0.5219, -0.7294,
        -2.2121, -0.4871,  0.0253,  0.3766])
tensor([ 0.0000,  0.0000, -3.3885, -0.0000,  0.0000,  0.6003,  0.4941,  0.0767,
         1.3666, -0.0000,  0.0000, -0.0000,  0.0000,  5.2249, -1.0437, -0.0000,
        -4.4242, -0.9743,  0.0507,  0.7532])


In [29]:

from collections import OrderedDict
new_check = OrderedDict()
for k, v in checkpoint.items():
    if 'module' not in k:
        k = 'module.'+k
    else:
        k = k.replace('features.module.', 'module.features.')
    new_check[k]=v

In [2]:
# # import torch
# # checkpoint = torch.load('./RAFDB/allepoch159.pkl')
# checkpoint['net']

In [26]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(

            # 1-1 conv layer
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            # 1-2 conv layer
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            # 1 Pooling layer
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = nn.Sequential(

            # 2-1 conv layer
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            # 2-2 conv layer
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            # 2 Pooling lyaer
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = nn.Sequential(

            # 3-1 conv layer
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            # 3-2 conv layer
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            # 3 Pooling layer
            # nn.MaxPool2d(kernel_size=2, stride=2))
            nn.AvgPool2d(kernel_size=24, stride=24))
        

        self.layer6 = nn.Sequential(

            # 6 Fully connected layer
            nn.Linear(256, 256),
            # nn.Dropout(),
            nn.ReLU())


        self.layer7 = nn.Sequential(

            # 7 Fully connected layer
            nn.Linear(256*10, 1024),
            # nn.Dropout(),
            nn.ReLU())

        self.layer8 = nn.Sequential(

            # 8 output layer
            nn.Linear(1024, 7),
            nn.Softmax())

    def forward(self, x):
        # x.shape ==(-1,10,96,96,3)
        out = []
        for k in range(10):
            #print(k)
            f = x[:,k,:,:,:]
            f = f.permute(0,3,1,2) 
            #（-1，3，96，96）
            f = self.layer1(f)  
            # (-1, 64, 48, 48)
            f = self.layer2(f)  
            # (-1, 128, 24, 24)
            f = self.layer3(f)  
            # (-1, 256, 1, 1)
            f = f.view(f.size(0),-1)
            # (-1, 256)
            f = self.layer6(f)
            # (-1, 256)
            out.append(f)
        out = torch.cat(out,axis=1) 
        #  (-1, 256*10)
        out = self.layer7(out)
        # (-1, 1024)
        out = self.layer8(out)
        # (-1, 7)
        return out
    
vgg16 = VGG16()

In [7]:
import torch.nn as nn
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.layer1 = []
        for i in range(10):
            self.layer1.append(nn.Sequential())
            self.layer1[i].add_module(str(i)+'conv1', nn.Conv2d(3, 64, kernel_size=3, padding=1))
            self.layer1[i].add_module(str(i)+'bn1', nn.BatchNorm2d(64))
            self.layer1[i].add_module(str(i)+'relu1',  nn.ReLU())
            self.layer1[i].add_module(str(i)+'conv2',  nn.Conv2d(64, 64, kernel_size=3, padding=1))
            self.layer1[i].add_module(str(i)+'bn2', nn.BatchNorm2d(64))
            self.layer1[i].add_module(str(i)+'relu2',  nn.ReLU())
            self.layer1[i].add_module(str(i)+'maxpool1',  nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = []
        for i in range(10):
            self.layer2.append(nn.Sequential())
            self.layer2[i].add_module(str(i)+'conv3', nn.Conv2d(64, 128, kernel_size=3, padding=1))
            self.layer2[i].add_module(str(i)+'bn3', nn.BatchNorm2d(128))
            self.layer2[i].add_module(str(i)+'relu3',  nn.ReLU())
            self.layer2[i].add_module(str(i)+'conv4',  nn.Conv2d(128, 128, kernel_size=3, padding=1))
            self.layer2[i].add_module(str(i)+'bn4', nn.BatchNorm2d(128))
            self.layer2[i].add_module(str(i)+'relu4',  nn.ReLU())
            self.layer2[i].add_module(str(i)+'maxpool2',  nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = []
        for i in range(10):
            self.layer3.append(nn.Sequential())
            self.layer3[i].add_module(str(i)+'conv5', nn.Conv2d(128, 256, kernel_size=3, padding=1))
            self.layer3[i].add_module(str(i)+'b53', nn.BatchNorm2d(256))
            self.layer3[i].add_module(str(i)+'relu5',  nn.ReLU())
            self.layer3[i].add_module(str(i)+'conv6',  nn.Conv2d(256 256, kernel_size=3, padding=1))
            self.layer3[i].add_module(str(i)+'bn6', nn.BatchNorm2d(256))
            self.layer3[i].add_module(str(i)+'relu6',  nn.ReLU())
            self.layer3[i].add_module(str(i)+'avgpool',  nn.AvgPool2d(kernel_size=24, stride=24))


    def forward(self, x):
        out = []
        for k in range(10):
            #print(k)
            f = x[:,k,:,:,:]
            f = f.permute(0,3,1,2) 
            f = self.layer1[k](f)  
            out.append(f)
        out = torch.cat(out,axis=1) 
        return out
        

In [12]:
vgg = VGG16()
vgg.layer1

[Sequential(
   (0conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (0bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (0relu1): ReLU()
   (0conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (0bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (0relu2): ReLU()
   (0maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 ), Sequential(
   (1conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (1bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (1relu1): ReLU()
   (1conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (1bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (1relu2): ReLU()
   (1maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 ), Sequential(
   (2conv1): Conv

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
# import torchvision.datasets as dsets
# import torchvision.transforms as transforms
from torch.autograd import Variable
import pickle

BATCH_SIZE =32
LEARNING_RATE = 0.01
EPOCH = 300

# transform = transforms.Compose([
#     transforms.RandomSizedCrop(224),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean = [ 0.485, 0.456, 0.406 ],
#                          std  = [ 0.229, 0.224, 0.225 ]),
#     ])



# with open('./ck+.pkl', 'rb') as f:
#      data = pickle.load(f)
        
# trainlabel = data[1][:820] 
# testlabel = data[1][820:]
# # dic = {'angry':0, 'disgust':1, 'fear':2, 'happy':3, 'neutral':4, 'sad':5, 'surprise':6}
# dic = {'angry':0, 'disgusted':1, 'fearful':2, 'happy':3, 'sadness':4, 'surprised':5}
# for i in range(len(testlabel)):
#     testlabel[i] = dic[testlabel[i]]
# for i in range(len(trainlabel)):
#     trainlabel[i] = dic[trainlabel[i]] 
    
# trainregion = data[3][:820]
# testregion = data[3][820:]

with open('./raf_train.pkl', 'rb') as f:
     train_data = pickle.load(f)
trainlabel = train_data[1]
trainregion = train_data[3]

with open('./raf_test.pkl', 'rb') as f:
     test_data = pickle.load(f)
testlabel = test_data[1]
testregion = test_data[3]

trainData = TensorDataset(torch.Tensor(trainregion), torch.LongTensor(trainlabel))
testData = TensorDataset(torch.Tensor(testregion), torch.LongTensor(testlabel))

trainLoader = DataLoader(dataset=trainData, batch_size=BATCH_SIZE, shuffle=True)
testLoader = DataLoader(dataset=testData, batch_size=1, shuffle=False)

class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.layer1 = []
        for i in range(10):
            self.layer1[i] = nn.Sequential()
            self.layer1[i].add_module(str(i)+'conv1', nn.Conv2d(3, 64, kernel_size=3, padding=1))
            self.layer1[i].add_module(str(i)+'bn1', nn.BatchNorm2d(64))
            self.layer1[i].add_module(str(i)+'relu1',  nn.ReLU())
            self.layer1[i].add_module(str(i)+'conv2',  nn.Conv2d(64, 64, kernel_size=3, padding=1))
            self.layer1[i].add_module(str(i)+'bn2', nn.BatchNorm2d(64))
            self.layer1[i].add_module(str(i)+'relu2',  nn.ReLU())
            self.layer1[i].add_module(str(i)+'maxpool',  nn.MaxPool2d(kernel_size=2, stride=2))
            

            

        self.layer2 = nn.Sequential(

            # 2-1 conv layer
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            # 2-2 conv layer
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            # 2 Pooling lyaer
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = nn.Sequential(

            # 3-1 conv layer
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            # 3-2 conv layer
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            # 3 Pooling layer
            # nn.MaxPool2d(kernel_size=2, stride=2))
            nn.AvgPool2d(kernel_size=24, stride=24))
        

        self.layer6 = nn.Sequential(

            # 6 Fully connected layer
            nn.Linear(256, 256),
            # nn.Dropout(),
            nn.ReLU())


        self.layer7 = nn.Sequential(

            # 7 Fully connected layer
            nn.Linear(256*10, 1024),
            # nn.Dropout(),
            nn.ReLU())

        self.layer8 = nn.Sequential(

            # 8 output layer
            nn.Linear(1024, 6),
            nn.Softmax())

    def forward(self, x):
        # x.shape ==(-1,18,12,12,3)
        #print(x.shape)
        out = []
        for k in range(10):
            #print(k)
            f = x[:,k,:,:,:]
            f = f.permute(0,3,1,2) 
            #（-1，3，96，96）
            f = self.layer1(f)  
            # (-1, 64, 48, 48)
            f = self.layer2(f)  
            # (-1, 128, 24, 24)
            f = self.layer3(f)  
            # (-1, 256, 1, 1)
            f = f.view(f.size(0),-1)
            # (-1, 256)
            f = self.layer6(f)
            # (-1, 256)
            out.append(f)
        out = torch.cat(out,axis=1) 
        #  (-1, 256*10)
        out = self.layer7(out)
        # (-1, 1024)
        out = self.layer8(out)
        # (-1, 6)
        return out
        


vgg16 = VGG16()
vgg16.cuda()

# Loss and Optimizer
cost = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg16.parameters(), lr=LEARNING_RATE)
# Train the model
for epoch in range(EPOCH):
    if epoch > 50:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.001
    if epoch > 150:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.0001
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(trainLoader):
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        out = vgg16.forward(images)
        #print(out.shape,labels.shape)
        loss = cost(out, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(out.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    
    print('epoch: %d' % epoch, loss.data)
    print('Test Accuracy of the model on the train images: %d %%' % (100 * correct / total))
    if (epoch+1) % 10 == 0 :
        torch.save(vgg16.state_dict(), './RAFDB/epoch{}.pkl'.format(epoch))
        
# Test the model
vgg16.eval()
correct = 0
total = 0

for images, labels in testLoader:
    images = Variable(images).cuda()
    outputs = vgg16(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()

print('Test Accuracy of the model on the test images: %d %%' % (100 * correct / total))

# Save the Trained Model


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
# import torchvision.datasets as dsets
# import torchvision.transforms as transforms
from torch.autograd import Variable
import pickle

BATCH_SIZE =32
LEARNING_RATE = 0.001
EPOCH = 100

# transform = transforms.Compose([
#     transforms.RandomSizedCrop(224),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean = [ 0.485, 0.456, 0.406 ],
#                          std  = [ 0.229, 0.224, 0.225 ]),
#     ])



with open('./ck+.pkl', 'rb') as f:
     data = pickle.load(f)
        
trainlabel = data[1][:820] 
testlabel = data[1][820:]
# dic = {'angry':0, 'disgust':1, 'fear':2, 'happy':3, 'neutral':4, 'sad':5, 'surprise':6}
dic = {'angry':0, 'disgusted':1, 'fearful':2, 'happy':3, 'sadness':4, 'surprised':5}
for i in range(len(testlabel)):
    testlabel[i] = dic[testlabel[i]]
for i in range(len(trainlabel)):
    trainlabel[i] = dic[trainlabel[i]] 
    
trainregion = data[3][:820]
testregion = data[3][820:]

trainData = TensorDataset(torch.Tensor(trainregion), torch.LongTensor(trainlabel))
testData = TensorDataset(torch.Tensor(testregion), torch.LongTensor(testlabel))

trainLoader = DataLoader(dataset=trainData, batch_size=BATCH_SIZE, shuffle=True)
testLoader = DataLoader(dataset=testData, batch_size=1, shuffle=False)

class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(

            # 1-1 conv layer
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            # 1-2 conv layer
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            # 1 Pooling layer
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = nn.Sequential(

            # 2-1 conv layer
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            # 2-2 conv layer
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            # 2 Pooling lyaer
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = nn.Sequential(

            # 3-1 conv layer
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            # 3-2 conv layer
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            # 3 Pooling layer
            # nn.MaxPool2d(kernel_size=2, stride=2))
            nn.AvgPool2d(kernel_size=24, stride=24))
        

        self.layer6 = nn.Sequential(

            # 6 Fully connected layer
            nn.Linear(256, 256),
            # nn.Dropout(),
            nn.ReLU())


        self.layer7 = nn.Sequential(

            # 7 Fully connected layer
            nn.Linear(256*10, 1024),
            # nn.Dropout(),
            nn.ReLU())

        self.layer8 = nn.Sequential(

            # 8 output layer
            nn.Linear(1024, 6),
            nn.Softmax())

    def forward(self, x):
        # x.shape ==(-1,18,12,12,3)
        f = x[:,k,:,:,:]
        f = f.permute(0,3,1,2) 
        #（-1，3，96，96）
        f = self.layer1(f)  
        # (-1, 64, 48, 48)
        f = self.layer2(f)  
        # (-1, 128, 24, 24)
        f = self.layer3(f)  
        # (-1, 256, 1, 1)
        f = f.view(f.size(0),-1)
        # (-1, 256)
        f = self.layer6(f)
        # (-1, 256)
        return f
        


vgg16 = []
vgg16.cuda()

# Loss and Optimizer
cost = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg16.parameters(), lr=LEARNING_RATE)
# Train the model
for epoch in range(EPOCH):
#     if epoch > 5:
#         optimizer.lr = 0.001
#         # print(optimizer.lr)
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(trainLoader):
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        out = []
        for k in range(10):
            vgg16.append(VGG16())
            out.append(vgg16[k].forward(images[:, k, :, :])) #(-1, 256)
        out = torch.cat(out,axis=1) 
        #  (-1, 256*10)
        out = vgg16[0].layer7(out)
        # (-1, 1024)
        out = vgg16[0].layer8(out)
        # (-1, 6)
        # out = vgg16.forward(images)
        #print(out.shape,labels.shape)
        loss = cost(out, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(out.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    
    print('epoch: %d' % epoch, loss.data)
    print('Test Accuracy of the model on the train images: %d %%' % (100 * correct / total))
    # if (epoch+1) % 10 == 0 :
        # print('Epoch [%d/%d], Loss. %.4f' %(epoch+1, EPOCH, loss.data[0]))
    #torch.save(vgg16.state_dict(), './epoch{}.pkl'.format(epoch))
# Test the model
vgg16.eval()
correct = 0
total = 0

for images, labels in testLoader:
    images = Variable(images).cuda()
    outputs = vgg16(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()

print('Test Accuracy of the model on the test images: %d %%' % (100 * correct / total))

# Save the Trained Model


In [8]:
import numpy as np
import torch
import torch.nn as nn
a = np.ones((1,12,1,1))
print(a.shape)
res = torch.FloatTensor([])
a = torch.FloatTensor(a)
b = torch.FloatTensor(np.ones((1,12,1,1)))
res = torch.cat((res, a), 1)
print(res.shape)
res = torch.cat((res, b), 1)
print(res.shape)

(1, 12, 1, 1)
torch.Size([1, 12, 1, 1])
torch.Size([1, 24, 1, 1])


In [ ]:
# -*- coding:utf-8 -*-
import torch
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
#数据预处理：转换为Tensor，归一化，设置训练集和验证集以及加载子进程数目
transform = transforms.Compose([transforms.ToTensor() , transforms.Normalize((0.5 , 0.5 , 0.5) , (0.5 , 0.5 , 0.5))])  #前面参数是均值，后面是标准差
trainset = torchvision.datasets.CIFAR10(root = './data' , train = True , download = True , transform = transform)
trainloader = torch.utils.data.DataLoader(trainset , batch_size = 4 , shuffle = True , num_workers =2) 
testset = torchvision.datasets.CIFAR10(root = './data' , train = False , download = True , transform = transform)
testloader = torch.utils.data.DataLoader(testset , batch_size = 4 , shuffle = True , num_workers = 2)
classes = ('plane' , 'car' , 'bird' , 'cat' , 'deer' , 'dog' , 'frog' , 'horse' , 'ship' , 'truck')


import matplotlib.pyplot as plt
import numpy as np
import pylab

def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg , (1 , 2 , 0)))
    pylab.show()

dataiter = iter(trainloader)
images , labels = dataiter.next()
for i in range(4):
    p = plt.subplot()
    p.set_title("label: %5s" % classes[labels[i]])
    imshow(images[i])
#构建网络
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net , self).__init__()
        self.conv1 = nn.Conv2d(3 , 6 , 5)
        self.pool = nn.MaxPool2d(2 , 2)
        self.conv2 = nn.Conv2d(6 , 16 , 5)
        self.fc1 = nn.Linear(16 * 5 * 5 , 120)
        self.fc2 = nn.Linear(120 , 84)
        self.fc3 = nn.Linear(84 , 10)

    def forward(self , x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1 , 16 * 5 * 5)  #利用view函数使得conv2层输出的16*5*5维的特征图尺寸变为400大小从而方便后面的全连接层的连接
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
net.cuda()

#define loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters() , lr = 0.001 , momentum = 0.9)

#train the Network
for epoch in range(2):
    running_loss = 0.0
    for i , data in enumerate(trainloader , 0):
        inputs , labels = data
        inputs , labels = Variable(inputs.cuda()) , Variable(labels.cuda())
        optimizer.zero_grad()
        #forward + backward + optimizer
        outputs = net(inputs)
        loss = criterion(outputs , labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.data[0]
        if i % 2000 == 1999:
            print('[%d , %5d] loss: %.3f' % (epoch + 1 , i + 1 , running_loss / 2000))
            running_loss = 0.0
print('Finished Training')

dataiter = iter(testloader)
images , labels = dataiter.next()
imshow(torchvision.utils.make_grid(images))
print('GroundTruth:' , ' '.join(classes[labels[j]] for j in range(4)))

outputs = net(Variable(images.cuda()))

_ , predicted = torch.max(outputs.data , 1)
print('Predicted: ' , ' '.join('%5s' % classes[predicted[j]] for j in range(4)))

correct = 0
total = 0
for data in testloader:
    images , labels = data
    outputs = net(Variable(images.cuda()))
    _ , predicted = torch.max(outputs.data , 1)
    correct += (predicted == labels.cuda()).sum()
    total += labels.size(0)
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

class_correct = torch.ones(10).cuda()
class_total = torch.ones(10).cuda()
for data in testloader:
    images , labels = data
    outputs = net(Variable(images.cuda()))
    _ , predicted = torch.max(outputs.data , 1)
    c = (predicted == labels.cuda()).squeeze()
    #print(predicted.data[0])
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (classes[i] , 100 * class_correct[i] / class_total[i]))

In [59]:
import torch
import torch.nn as nn
import pickle
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Variable
import numpy as np
from IPython import embed
import torch.nn.functional as F

with open('./raf_test_96.pkl', 'rb') as f:
     test_data = pickle.load(f)
print('load test done')

# testData = TensorDataset((torch.Tensor(test_data[2]))[:, :10], torch.LongTensor(test_data[0]))
# # testData = TensorDataset(torch.Tensor(test_data[2]), torch.LongTensor(test_data[0]))
# print('to tensor done')

# testLoader = DataLoader(dataset=testData, batch_size=1, shuffle=False)
# print('dataloader done')




load test done


In [60]:
# testData = TensorDataset(torch.Tensor(test_data[2]), torch.LongTensor(test_data[0]))
testData = TensorDataset((torch.Tensor(test_data[2]))[:, :10], torch.LongTensor(test_data[0]))
print('to tensor done')

testLoader = DataLoader(dataset=testData, batch_size=1, shuffle=False)
print('dataloader done')

to tensor done
dataloader done


In [74]:
class VGG16(nn.Module):

    def __init__(self, num_classes=7):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256*2*2*10, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, num_classes),
            # nn.Softmax()
        )

    def forward(self, x):
        out = []
        for k in range(10):
            f = x[:,k,:,:,:]
            f = f.permute(0,3,1,2) 
            f = self.features(f)
            f = f.view(f.size(0), 256 * 2 * 2)
            out.append(f)
        out = torch.cat(out, axis=1)
        out = self.classifier(out)
        return out

In [75]:
vgg16 = VGG16()
# vgg16 = torch.nn.DataParallel(vgg16)

path = './RAFDB/alexnetnewepoch169.pkl'
from collections import OrderedDict
state_dictBA = torch.load(path)['net']
# create new OrderedDict that does not contain `module.`
new_state_dictBA = OrderedDict()
for k, v in state_dictBA.items():
    name = k[7:] # remove `module.`
    new_state_dictBA[name] = v
vgg16.load_state_dict(new_state_dictBA)
# checkpoint = torch.load(path)

# vgg16.load_state_dict(checkpoint['net'])
vgg16.cuda()

# Test the model
vgg16.eval()

correct = list(0. for i in range(7))
total = list(0. for i in range(7))
suprise = list(0. for i in range(7))
fear = list(0. for i in range(7))
disgust = list(0. for i in range(7))
happy = list(0. for i in range(7))
sad = list(0. for i in range(7))
anger = list(0. for i in range(7))
netual = list(0. for i in range(7))
for i, (images, labels) in enumerate(testLoader):
    images = Variable(images.cuda())
    labels = Variable(labels.cuda())

    output = vgg16(images)

    prediction = torch.argmax(output, 1)
    res = prediction == labels
    
    for label_idx in range(len(labels)):  
        pre = prediction[label_idx]
        label_single = labels[label_idx]
        if label_single == 0:
            suprise[pre] += 1
        elif label_single == 1:
            fear[pre] += 1
        elif label_single == 2:
            disgust[pre] += 1
        elif label_single == 3:
            happy[pre] += 1
        elif label_single == 4:
            sad[pre] += 1
        elif label_single == 5:
            anger[pre] += 1
        elif label_single == 6:
            netual[pre] += 1
        
        correct[label_single] += res[label_idx].item()
        total[label_single] += 1
acc_str = 'Accuracy: %f'%(sum(correct)/sum(total))
print(correct)
print(total)
for acc_idx in range(7):
    try:
        acc = correct[acc_idx]/total[acc_idx]
        print(acc)
    except:
        acc = 0
    finally:
        acc_str += '\tclassID:%d\tacc:%f\t'%(acc_idx+1, acc)
print(acc_str)


[265.0, 29.0, 56.0, 1044.0, 313.0, 96.0, 512.0]
[329.0, 74.0, 160.0, 1185.0, 478.0, 162.0, 680.0]
0.8054711246200608
0.3918918918918919
0.35
0.8810126582278481
0.6548117154811716
0.5925925925925926
0.7529411764705882
Accuracy: 0.754563	classID:1	acc:0.805471		classID:2	acc:0.391892		classID:3	acc:0.350000		classID:4	acc:0.881013		classID:5	acc:0.654812		classID:6	acc:0.592593		classID:7	acc:0.752941	


In [83]:
netual

[22.0, 3.0, 15.0, 68.0, 56.0, 4.0, 512.0]

In [84]:
# alexnet 
(0.8054711246200608+0.3918918918918919+0.35+0.8810126582278481+0.6548117154811716+0.5925925925925926+0.7529411764705882)/7


0.6326744513263076

In [76]:
(0.8054711246200608+0.3918918918918919+0.35+0.8810126582278481+0.6548117154811716+0.5925925925925926+0.7529411764705882)/7

0.6326744513263076